In [3]:
import pandas as pd
import numpy as np
# import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/all_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels2021.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels2020.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno
docnos21 = qrels21.docno
docnos20 = qrels20.docno

In [6]:
import os
# rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100"

objective="ndcg_cut_5"
for i, (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
            tuned_bm25, parameters = pt.KFoldGridSearch(
                bm25,
                {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                "bm25.k_3": [2, 5, 8, 10]
                }},
                [topics20, topics21],
                [qrels20, qrels21],
                objective
            )
            print(pt.Experiment([tuned_bm25, bm25], topics, qrels, 
                                names=['tuned_bm25','orignal_bm25'],
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            print(pt.Experiment([tuned_bm25, bm25], topics20, qrels, names=['20original_tuned_bm25', '20orignal_bm25'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            
            print(pt.Experiment([tuned_bm25, bm25], topics21, qrels, names=['21original_tuned_bm25', '21orignal_bm25'], 
                                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]))
            tuned_bm25.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/original_tuned_bm.csv")
#             tuned_bm25.to_csv(path_or_buf=f"/workspace/src/runs_bm25/original_tuned_bm25.csv")
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_original-100
Fold 1
Best ndcg_cut_5 is 0.683724
Best setting is ['BR(BM25) c=0.55', 'BR(BM25) bm25.k_1=2.2', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.526937
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2']
           name  ndcg_cut_5  ndcg_cut_25     bpref
0    tuned_bm25    0.519366     0.446745  0.632032
1  orignal_bm25    0.433057     0.384884  0.634694
                    name  ndcg_cut_5  ndcg_cut_25     bpref
0  20original_tuned_bm25    0.380818     0.346446  0.642787
1         20orignal_bm25    0.308863     0.307938  0.643398
                    name  ndcg_cut_5  ndcg_cut_25     bpref
0  21original_tuned_bm25    0.655142     0.545039  0.621493
1         21orignal_bm25    0.554767     0.460291  0.626163


In [7]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
folders = ['conclusions','pool_topics','topics','topics_conclusion','topics_conclusion_leakage','topics_leakage','pool_topics_conclusions','conclusions_leakage','pools_conclusions']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
            tuned_bm25, _ = pt.KFoldGridSearch(
                bm25,
                {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                "bm25.k_3": [2, 5, 8, 10]
                }},
                [topics20, topics21],
                [qrels20, qrels21],
                metrics
            )
            print(pt.Experiment([tuned_bm25, bm25], topics, qrels, names=[folders[i] + '_tuned_bm25', folders[i] + '_orignal_bm25'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_bm25, bm25], topics20, qrels, names=['20' + folders[i] + 'tuned_bm25', '20' + folders[i] + '_bm25'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_bm25, bm25], topics21, qrels, names=['21' + folders[i] + 'tuned_bm25', '21'+ folders[i] + '_bm25'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            tuned_bm25.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_bm25.csv")
#             tuned_bm25.to_csv(path_or_buf=f"/workspace/src/runs_bm25/{folders[i]}_tuned_bm25.csv")
            i += 1
            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
Fold 1
Best ndcg_cut_5 is 0.580154
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.405992
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=4.4', 'BR(BM25) bm25.k_3=2']


/tmp/ipykernel_3684168/494468316.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25, bm25], topics, qrels, names=[folders[i] + '_tuned_bm25', folders[i] + '_orignal_bm25'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                      name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    conclusions\_tuned\_bm25 &    0.493952 &     0.441997 &  0.717687 \\
1 &  conclusions\_orignal\_bm25 &    0.394756 &     0.375585 &  0.710391 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3684168/494468316.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25, bm25], topics20, qrels, names=['20' + folders[i] + 'tuned_bm25', '20' + folders[i] + '_bm25'],


\begin{tabular}{llrrr}
\toprule
{} &                     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20conclusionstuned\_bm25 &    0.405992 &     0.363616 &  0.703588 \\
1 &       20conclusions\_bm25 &    0.305337 &     0.304660 &  0.695431 \\
\bottomrule
\end{tabular}



...

\begin{tabular}{llrrr}
\toprule
{} &                            name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    pools\_conclusions\_tuned\_bm25 &    0.486016 &     0.433605 &  0.715633 \\
1 &  pools\_conclusions\_orignal\_bm25 &    0.396597 &     0.372488 &  0.708422 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3684168/494468316.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25, bm25], topics20, qrels, names=['20' + folders[i] + 'tuned_bm25', '20' + folders[i] + '_bm25'],


\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20pools\_conclusionstuned\_bm25 &    0.377257 &     0.345495 &  0.703909 \\
1 &       20pools\_conclusions\_bm25 &    0.294971 &     0.298600 &  0.698183 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3684168/494468316.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25, bm25], topics21, qrels, names=['21' + folders[i] + 'tuned_bm25', '21'+ folders[i] + '_bm25'],


\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21pools\_conclusionstuned\_bm25 &    0.592600 &     0.519952 &  0.727122 \\
1 &       21pools\_conclusions\_bm25 &    0.496191 &     0.444897 &  0.718457 \\
\bottomrule
\end{tabular}



In [8]:
'''
Example of simple tuning bm25 and its not saving to the folder 
'''

import os
rootdir = "/workspace/src/grid_search_right_qrels/super_folder/"
folders = ['marco-pretrained']
metrics="ndcg_cut_5"
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            path = subdir + '/'+ file
            print('\n' + subdir)
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            bm25 = pt.BatchRetrieve(index, wmodel="BM25", controls={"c" : 0.75, "bm25.k_1": 1.2, "bm25.k_3": 8})
            tuned_bm25, _ = pt.KFoldGridSearch(
                bm25,
                {bm25: {"c" : [0.15, 0.35, 0.55, 0.75],
                "bm25.k_1": [0.6, 1.2, 1.6, 2.2, 2.8, 3.4, 4.0, 4.4],
                "bm25.k_3": [2, 5, 8, 10]
                }},
                [topics20, topics21],
                [qrels20, qrels21],
                metrics
            )
            print(pt.Experiment([tuned_bm25, bm25], topics, qrels, names=[folders[i] + '_tuned_bm25', folders[i] + '_orignal_bm25'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_bm25, bm25], topics20, qrels, names=['20' + folders[i] + 'tuned_bm25', '20' + folders[i] + '_bm25'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            
            print(pt.Experiment([tuned_bm25, bm25], topics21, qrels, names=['21' + folders[i] + 'tuned_bm25', '21'+ folders[i] + '_bm25'], 
                eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            tuned_bm25.to_csv(path_or_buf=f"/workspace/src/grid_search_right_qrels/final_right/run/{folders[i]}_tuned_bm25.csv")
#             tuned_bm25.to_csv(path_or_buf=f"/workspace/src/runs_bm25/{folders[i]}_tuned_bm25.csv")
            i += 1
            


/workspace/src/grid_search_right_qrels/super_folder/
Fold 1
Best ndcg_cut_5 is 0.460400
Best setting is ['BR(BM25) c=0.15', 'BR(BM25) bm25.k_1=3.4', 'BR(BM25) bm25.k_3=2']
Fold 2
Best ndcg_cut_5 is 0.278232
Best setting is ['BR(BM25) c=0.75', 'BR(BM25) bm25.k_1=0.6', 'BR(BM25) bm25.k_3=2']


/tmp/ipykernel_3684168/3015958913.py:28: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25, bm25], topics, qrels, names=[folders[i] + '_tuned_bm25', folders[i] + '_orignal_bm25'], eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())


\begin{tabular}{llrrr}
\toprule
{} &                           name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &    marco-pretrained\_tuned\_bm25 &    0.329196 &     0.304758 &  0.622564 \\
1 &  marco-pretrained\_orignal\_bm25 &    0.335913 &     0.305915 &  0.620544 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3684168/3015958913.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25, bm25], topics20, qrels, names=['20' + folders[i] + 'tuned_bm25', '20' + folders[i] + '_bm25'],


\begin{tabular}{llrrr}
\toprule
{} &                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  20marco-pretrainedtuned\_bm25 &    0.257784 &     0.261112 &  0.621228 \\
1 &       20marco-pretrained\_bm25 &    0.272867 &     0.259365 &  0.618440 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3684168/3015958913.py:33: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(pt.Experiment([tuned_bm25, bm25], topics21, qrels, names=['21' + folders[i] + 'tuned_bm25', '21'+ folders[i] + '_bm25'],


\begin{tabular}{llrrr}
\toprule
{} &                          name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  21marco-pretrainedtuned\_bm25 &    0.399179 &     0.347531 &  0.623873 \\
1 &       21marco-pretrained\_bm25 &    0.397697 &     0.351533 &  0.622606 \\
\bottomrule
\end{tabular}

